In [64]:
import pandas as pd
import numpy as np

# Adjust notebook settings to widen the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important;}</style>"))
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [65]:
hpsa = pd.read_csv("./Data/HPSA_Cleaned.csv")
nsduh = pd.read_csv("./Data/df_subset.csv")
grants = pd.read_csv("./Data/grants_per_county_cbsa.csv")

/Users/chiufengyap/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (133) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [66]:
cbsa = pd.read_csv('./Data/cbsa2fipsxw.csv')
cbsa.head(2)

,cbsacode,metrodivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33860.0,NaN,NaN,"Montgomery, AL",Metropolitan Statistical Area,NaN,NaN,Autauga County,Alabama,1.0,1.0,Central


In [67]:
cbsa_data = cbsa[['countycountyequivalent', 'centraloutlyingcounty', 'statename', 'cbsacode','metropolitanmicropolitanstatis']]
cbsa_grant = grants.merge(cbsa_data, how='left', left_on=['countycountyequivalent','statename', 'centraloutlyingcounty', 'cbsacode'], right_on=['countycountyequivalent','statename', 'centraloutlyingcounty', 'cbsacode'])
cbsa_grant.head()

,Complete County Name,State Name,Total Active Grant Financial Assistance,Mental Health Assistance,countycountyequivalent,centraloutlyingcounty,statename,cbsacode,Outlying,metropolitanmicropolitanstatis
0,Ada County,Idaho,4940351.0,225000.0,Ada County,Central,Idaho,14260.0,0,Metropolitan Statistical Area
1,Adair County,Missouri,3204629.0,0.0,Adair County,Central,Missouri,28860.0,0,Micropolitan Statistical Area
2,Adams County,Colorado,419040.0,0.0,Adams County,Central,Colorado,19740.0,0,Metropolitan Statistical Area
3,Adams County,Nebraska,100000.0,100000.0,Adams County,Central,Nebraska,25580.0,0,Micropolitan Statistical Area
4,Adams County,Washington,6814261.0,0.0,Adams County,Central,Washington,36830.0,0,Micropolitan Statistical Area


In [68]:
# add PDEN10 Mapper 
cbsa_grant['PDEN10'] = np.where(cbsa_grant['metropolitanmicropolitanstatis'] == 'Metropolitan Statistical Area',1,
                                np.where(cbsa_grant['metropolitanmicropolitanstatis'] == 'Micropolitan Statistical Area',
                                2,3))
cbsa_grant['PDEN10'].value_counts()

1    513
2    185
Name: PDEN10, dtype: int64

In [69]:
## save merged grant and cbsa dataset in ./Data folder 
cbsa_grant.to_csv('./Data/Grant_Cleaned.csv',index=False)

In [72]:
# merge HPSA and Grant by CBSA identifiers
HPSA_Grant = hpsa.merge(cbsa_grant, how='left', left_on=['countycountyequivalent','centraloutlyingcounty', 'statename', 'cbsacode', 'metropolitanmicropolitanstatis', 'PDEN10', 'State Name'], right_on=['countycountyequivalent','centraloutlyingcounty', 'statename', 'cbsacode', 'metropolitanmicropolitanstatis', 'PDEN10', 'State Name'])

In [73]:
HPSA_Grant.head(2)

,Withdrawn Date,HPSA Name,Common State County FIPS Code,Common State FIPS Code,Common State Name,County Equivalent Name,HPSA Component Name,HPSA Component Type Code,HPSA Component Type Description,HPSA Designation Population Type Description,Common State Abbreviation,HPSA Metropolitan Indicator Code,HPSA Type Code,Primary State FIPS Code,Primary State Name,State Abbreviation,State and County Federal Information Processing Standard Code,State FIPS Code,State Name,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,HPSA Status Code,Common Region Name,County or County Equivalent Federal Information Processing Standard Code,Common County Name,Metropolitan Indicator,HPSA Designation Last Update Date,HPSA Designation Date,HPSA Status,HPSA Geography Identification Number,HPSA Score,Primary State Abbreviation,Primary HHS Region Name,Designation Type,HPSA ID,HPSA Designation Population,Rural Status,Rural Status Code,HPSA Degree of Shortage,HPSA FTE,HPSA Component State Abbreviation,HPSA Population Type,HPSA Population Type Code,HPSA Shortage,DaysBeforeWithdrawn,County Equivalent Name New,countycountyequivalent,centraloutlyingcounty,statename,cbsacode,metropolitanmicropolitanstatis,PDEN10,Complete County Name,Total Active Grant Financial Assistance,Mental Health Assistance,Outlying
0,NaN,Stanley Correctional Institution,55017,55,Wisconsin,Chippewa,Stanley Correctional Institution,UNK,Unknown,Correctional Facility,WI,0,PRSN,55,Wisconsin,WI,55017,55,Wisconsin,N,N,D,Region 5,017,"Chippewa County, WI",Unknown,08/02/2018,07/21/2003,Designated,POINT,15,WI,Region 5,Correctional Facility,7551065910,2885.0,Non-Rural,N,6,0.6,NaN,NaN,NaN,0.84,NaN,Chippewa County,Chippewa County,Central,Wisconsin,20740.0,Metropolitan Statistical Area,1,NaN,NaN,NaN,NaN
1,07/02/2018,Rock County,55105,55,Wisconsin,Rock,Rock,SCTY,Single County,Geographic Population,WI,0,Hpsa Geo HN,55,Wisconsin,WI,55105,55,Wisconsin,N,N,W,Region 5,105,"Rock County, WI",Unknown,07/02/2018,04/09/2014,Withdrawn,55105,13,WI,Region 5,High Needs Geographic HPSA,755105,156639.0,Partially Rural,P,Not applicable,8.5,WI,Geographic Population,TRC,0.74,1545 days,Rock County,Rock County,Central,Wisconsin,27500.0,Metropolitan Statistical Area,1,Rock County,86677.0,0.0,0.0


In [74]:
## save merged HPSA and Grant by CBSA columns dataset in ./Data folder 
HPSA_Grant.to_csv('./Data/hpsa_grant_cbsa.csv',index=False)